## Building Applications with ChatGPT O1-preview

In this code-along, we will build interactive dashboards & visualizations that help us understand the stock market better. We will build a system that:

*   For given stocks, we understand its stock price
*   For given stocks, we understand its stock price

## Instructions to prompt-along

1. You need to be logged into gmail
2. Press on `File > Save a copy in Drive`
3. You will have your own copy of this notebook


## Step 1: Build a stock price visualization tool

_Prompt Used: I would like to build a stock price visualization tool using Python. The outcome of this application needs to be an interactive visualization that is embedded in a Google Colab, where there are two drop downs, one for a given stock, and one for different time horizons (1M, 3M, 6M, etc..). Use matplotlib and seaborn for the visualizations, and Yahoo Finance to access the data. Please provide me detailed instructions on how I can paste the code into Google Colab._

In [ ]:
# Install yfinance
!pip install yfinance --quiet

In [ ]:
# Import libraries
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display
import ipywidgets as widgets

In [ ]:
# List of stock symbols
stock_options = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

# Dictionary of time horizons
time_horizons = {
    '1 Month': 30,
    '3 Months': 90,
    '6 Months': 180,
    '1 Year': 365,
    '2 Years': 730
}

# Create dropdown widgets
stock_dropdown = widgets.Dropdown(
    options=stock_options,
    value='AAPL',
    description='Stock:',
)

time_dropdown = widgets.Dropdown(
    options=time_horizons.keys(),
    value='1 Month',
    description='Time Horizon:',
)

In [ ]:
def plot_stock(stock_symbol, time_period):
    # Calculate the start date based on the time horizon
    end_date = datetime.today()
    delta_days = time_horizons[time_period]
    start_date = end_date - timedelta(days=delta_days)

    # Fetch stock data
    data = yf.download(stock_symbol, start=start_date, end=end_date)

    # Reset index to get 'Date' as a column
    data.reset_index(inplace=True)

    # Plot using seaborn
    sns.set(style='white')  # Removed grid lines by changing the style
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=data,
        x='Date',
        y='Close',
        marker='o',
        linewidth=2.5  # Made the line thicker
    )
    plt.title(f'{stock_symbol} Stock Price ({time_period})', loc='left')  # Aligned title to the left
    plt.xlabel('Date')
    plt.ylabel('Closing Price')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# Create interactive plot
interactive_plot = widgets.interactive(
    plot_stock,
    stock_symbol=stock_dropdown,
    time_period=time_dropdown
)

# Display widgets and output
display(interactive_plot)

interactive(children=(Dropdown(description='Stock:', index=2, options=('AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'…

## Step 2: Refine the stock price visualization tool, add custom date pickers, and more indicators.

_Prompt Used: Can you now help me update this script, by adding date pickers for custom data rangers, and a drop down for different indicators to visualize? Please provide a simple moving average, exponential moving average, and bollinger bands as the additional indicators._

1. Simple Moving Average (SMA)
Definition: The Simple Moving Average calculates the average of a selected range of prices, usually closing prices, by the number of periods in that range.
Purpose: It smooths out price data to identify the trend direction over a period.
Usage: Traders use SMA to determine if the asset price will continue or reverse a bull or bear trend.
2. Exponential Moving Average (EMA)
Definition: The Exponential Moving Average gives more weight to recent prices, making it more responsive to new information.
Purpose: EMA reacts faster to recent price changes compared to SMA.
Usage: Traders use EMA to spot price trends, as it can signal a change in the market earlier than SMA.
3. Bollinger Bands
Definition: Bollinger Bands consist of a middle band (usually a 20-day SMA) and two outer bands set two standard deviations above and below the middle band.
Purpose: They measure market volatility and provide a high and low range within which a security typically trades.
Usage: Traders use Bollinger Bands to identify overbought or oversold conditions and to predict possible price reversals.

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display
import ipywidgets as widgets
from matplotlib.dates import DateFormatter

In [ ]:
# List of stock symbols
stock_options = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

# Create stock dropdown widget
stock_dropdown = widgets.Dropdown(
    options=stock_options,
    value='AAPL',
    description='Stock:',
)

# Create date picker widgets
start_date_picker = widgets.DatePicker(
    description='Start Date',
    value=datetime.today() - timedelta(days=180),  # Default to 6 months ago
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    value=datetime.today(),
)

# Indicator options
indicator_options = [
    'Closing Price',
    'Simple Moving Average',
    'Exponential Moving Average',
    'Bollinger Bands'
]

# Create indicator dropdown
indicator_dropdown = widgets.Dropdown(
    options=indicator_options,
    value='Closing Price',
    description='Indicator:',
)

In [ ]:
def plot_stock(stock_symbol, start_date, end_date, indicator):
    # Ensure end_date is after start_date
    if start_date > end_date:
        print("Error: Start date must be before end date.")
        return

    # Fetch stock data
    data = yf.download(stock_symbol, start=start_date, end=end_date)

    if data.empty:
        print("No data available for the selected date range.")
        return

    # Reset index to get 'Date' as a column
    data.reset_index(inplace=True)

    # Compute Indicators
    if indicator == 'Simple Moving Average':
        data['SMA'] = data['Close'].rolling(window=20).mean()
    elif indicator == 'Exponential Moving Average':
        data['EMA'] = data['Close'].ewm(span=20, adjust=False).mean()
    elif indicator == 'Bollinger Bands':
        data['SMA'] = data['Close'].rolling(window=20).mean()
        data['StdDev'] = data['Close'].rolling(window=20).std()
        data['Upper Band'] = data['SMA'] + (data['StdDev'] * 2)
        data['Lower Band'] = data['SMA'] - (data['StdDev'] * 2)

    # Plot using seaborn
    sns.set(style='white')  # Style without grid lines
    plt.figure(figsize=(12, 6))

    if indicator == 'Closing Price':
        sns.lineplot(data=data, x='Date', y='Close', marker='o', linewidth=2.5)
    elif indicator == 'Simple Moving Average':
        sns.lineplot(data=data, x='Date', y='Close', label='Closing Price', linewidth=1)
        sns.lineplot(data=data, x='Date', y='SMA', label='SMA (20)', linewidth=2.5)
    elif indicator == 'Exponential Moving Average':
        sns.lineplot(data=data, x='Date', y='Close', label='Closing Price', linewidth=1)
        sns.lineplot(data=data, x='Date', y='EMA', label='EMA (20)', linewidth=2.5)
    elif indicator == 'Bollinger Bands':
        sns.lineplot(data=data, x='Date', y='Close', label='Closing Price', linewidth=1)
        sns.lineplot(data=data, x='Date', y='Upper Band', label='Upper Band', linestyle='--', linewidth=1.5)
        sns.lineplot(data=data, x='Date', y='Lower Band', label='Lower Band', linestyle='--', linewidth=1.5)

    plt.title(f'{stock_symbol} {indicator} ({start_date.strftime("%Y-%m-%d")} to {end_date.strftime("%Y-%m-%d")})', loc='left')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Create interactive plot
interactive_plot = widgets.interactive(
    plot_stock,
    stock_symbol=stock_dropdown,
    start_date=start_date_picker,
    end_date=end_date_picker,
    indicator=indicator_dropdown
)

# Display widgets and output
display(interactive_plot)

interactive(children=(Dropdown(description='Stock:', options=('AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'), value=…

## Step 3: Build a financial news sentiment analysis bot.

_Prompt Used: Now can you build a simple financial news sentiment analysis bot that analyzes headlines from financial news articles using the [Finnhub API](https://finnhub.io/docs/api). The outcome needs to be a DataFrame with 4 columns, the stock, the date, title of the article, and the sentiment._

**NOTE: Finnhub's free API for news returns lets you analyze one stock and 5 news stories only in your given time period — you need premium access to get all the stocks and news stories.**

Create a free API key by:

1. In a new browser tab, go to https://finnhub.io/
2. Click on "Get a free API key"
3. Enter your name, email, and password, and click register.
4. Copy the API key.
5. Return to your Colab notebook and use the left-hand navigation bar to open the Secrets pane.
6. Name your API key "finnhubAPI".
7. Paste the API key into the value textbox.
8. Change the Notebook Access slider to "on".

```py
from google.colab import userdata
finnhub_API = userdata.get('finnhubAPI')
```

In [ ]:
# Install necessary libraries
!pip install requests --quiet
!pip install nltk --quiet

# Download NLTK data
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
import requests
import pandas as pd
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
# Replace 'YOUR_FINNHUB_API_KEY' with your actual Finnhub API key
from google.colab import userdata
finnhub_API = userdata.get('finnhubAPI')

In [ ]:
def fetch_news(stock_symbol):
    from_date = (datetime.today() - pd.DateOffset(days=30)).strftime('%Y-%m-%d')  # Last 30 days
    to_date = datetime.today().strftime('%Y-%m-%d')
    url = f'https://finnhub.io/api/v1/company-news?symbol={stock_symbol}&from={from_date}&to={to_date}&token={finnhub_API}'
    response = requests.get(url)
    if response.status_code == 200:
        news = response.json()
        return news
    else:
        print(f"Failed to fetch news for {stock_symbol}. HTTP Status code: {response.status_code}")
        return []

In [ ]:
def analyze_sentiment(headline):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(headline)
    sentiment = sentiment_score['compound']
    return sentiment


In [ ]:
def get_news_sentiment(stock_symbol):
    news = fetch_news(stock_symbol)
    data = []
    for article in news:
        date = datetime.fromtimestamp(article['datetime']).strftime('%Y-%m-%d')
        title = article['headline']
        sentiment = analyze_sentiment(title)
        data.append({
            'Stock': stock_symbol,
            'Date': date,
            'Title': title,
            'Sentiment': sentiment
        })
    df = pd.DataFrame(data)
    return df


In [ ]:
# Example usage
stock_symbol = 'AAPL'  # You can change this to any stock symbol
df = get_news_sentiment(stock_symbol)
df

,Stock,Date,Title,Sentiment
0,AAPL,2024-10-02,Just Do It! Brand Name Lessons From Nike's Tro...,-0.5093
1,AAPL,2024-10-02,The top stock sectors to watch for the rest of...,0.2023
2,AAPL,2024-10-01,Why Arm Holdings Stock Lost 5% on Tuesday,-0.3182
3,AAPL,2024-10-01,How to play tech amid recent volatility,0.3400
4,AAPL,2024-10-01,"Apple readies iPhone SE, iPad Air upgrades for...",0.0000
...,...,...,...,...
218,AAPL,2024-09-27,Apple’s $3.3 Trillion “Self-Destructive” Secret…,0.0000
219,AAPL,2024-09-27,Apple's Siri integration monetization potentia...,0.0000
220,AAPL,2024-09-27,"Judge narrows privacy lawsuit against Apple, R...",-0.2263
221,AAPL,2024-09-27,What You Missed On Wall Street On Friday,-0.2960
